# Fama-French 5 factor Model Analysis Tool

### **references :**
1. https://ssupapers.tistory.com/1
2. ChatGPT

<br>

- 원본 3 factor 코드(PAPERS tistory 블로그 제공) 활용.
- RMSE 도출하는 로직 추가.
- 두 가지 추가 팩터인 RMW (Profitability)와 CMA (Investment)도 분석에 포함.

---

# I. 필요 라이브러리 호출 및 함수 정의

In [1]:
import yfinance as yf
import pandas_datareader.data as web
import pandas as pd
import statsmodels.api as sm
import numpy as np

# 월말 수익률 도출 메소드
def get_monthly_returns(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start = start_date, end = end_date, interval = '1mo')

    x = stock_data['Adj Close'].resample('M').last().pct_change().dropna()

    x.name = 'Return' # = 수익률

    return x

# 파마-프렌치 5 factor 데이터 가져오는 메소드
def get_factors(start_date, end_date):
    # pandas datareader를 사용하여 F-F_Research_Data_5_Factors_2x3 데이터를 가져옴
    ff_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=start_date, end=end_date)

    # 데이터는 딕셔너리 형태이므로 첫 번째 데이터프레임을 추출
    ff_data = ff_data[0]

    # PeriodIndex를 Timestamp로 변환
    ff_data.index = ff_data.index.to_timestamp()

    # 필요한 컬럼만 선택하여 월 단위로 리샘플링
    ff_data = ff_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']].resample('M').last()

    # 퍼센트 단위를 소수점으로 변환
    ff_data /= 100

    return ff_data

# 수익률과 팩터 데이터를 하나의 데이터프레임으로 병합하는 메소드
def merge_data(ticker, start_date, end_date):
    monthly_returns = get_monthly_returns(ticker, start_date, end_date)

    fama_french = get_factors(start_date, end_date)

    df = pd.merge(monthly_returns, fama_french, left_index=True, right_index=True)

    df['Excess Return'] = df['Return'] - df['RF'] # Excess Return : 초과수익률 의미

    return df

# 파마-프렌치 5 factor 모델로 OLS 회귀를 진행하여 타겟변수 ETF에 대해 분석하는 메소드
def estimate_ff_model(df):
    X = df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

    # 독립변수 X : 5 factors
    X = sm.add_constant(X) # 모델에 상수항(alpha = y절편)을 추가

    # 종속변수 y : 분석하고자 하는 ETF의 (무위험채권 대비) 초과수익률
    y = df['Excess Return']

    # OLS(최소자승법)을 사용해 회귀모델 적합
    model = sm.OLS(y, X).fit()

    # 예측값 계산
    predictions = model.predict(X)

    # RMSE 계산
    rmse = np.sqrt(((predictions - y) ** 2).mean())

    return model.summary(), rmse

---

# II. 각 ETF 분석 수행
- User에게 직접 ETF ticker명 이름을 input으로 받도록 수정.


In [ ]:
def main():
    # ticker = input('분석하고자 하는 ETF의 ticker명 입력(Yahoo Finance 기준) : ')
    ticker_list = ['IWM', 'SPY', 'VTV', 'VUG', 'MTUM', 'QUAL', 'VYMI', 'USMV', 'KBE', 'IYK', 'IYC']

    start_date = '2000-01-01'
    end_date = '2024-09-01'

    for ticker in ticker_list:
        df = merge_data(ticker, start_date, end_date)

        model_summary, rmse = estimate_ff_model(df)

        print(f'\n{model_summary}\n') ; print(f'\nRMSE : {rmse}\n')

if __name__ == "__main__":
    main()

| ETF | RMSE | R-squared | Adjusted R-squared | Constant | Mkt-RF | SMB | HML | RMW | CMA |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| IWM | 0.00700 | 0.986 | 0.985 | -0.0008 | 0.9975 | 0.8176 | 0.0707 | -0.0732 | 0.0111 |
| SPY | 0.00532 | 0.986 | 0.986 | -0.0003 | 1.0092 | -0.1647 | 0.0353 | 0.0533 | 0.0196 |
| VTV | 0.00910 | 0.956 | 0.955 | -0.0005 | 0.9366 | -0.1453 | 0.3174 | 0.0562 | 0.1410 |
| VUG | 0.00818 | 0.971 | 0.970 | 0.0002 | 1.0810 | -0.1193 | -0.2490 | 0.0256 | -0.1540 |
| MTUM | 0.01847 | 0.835 | 0.828 | 0.0009 | 0.9927 | -0.2539 | -0.1654 | -0.2162 | 0.1566 |
| QUAL | 0.00730 | 0.973 | 0.972 | -0.0007 | 1.0032 | -0.1250 | -0.0200 | 0.1877 | -0.0280 |
| VYMI | 0.02291 | 0.762 | 0.749 | -0.0030 | 0.8123 | -0.0730 | 0.3713 | 0.0085 | 0.0988 |
| USMV | 0.01376 | 0.834 | 0.829 | -0.0001 | 0.7547 | -0.1352 | -0.1116 | 0.2627 | 0.2725 |
| KBE | 0.03454 | 0.814 | 0.810 | 0.0011 | 0.9388 | 0.1982 | 1.4944 | -0.6021 | -0.8515 |
| IYK | 0.02022 | 0.717 | 0.712 | -0.0011 | 0.7708 | -0.0480 | -0.0427 | 0.4101 | 0.4777 |
| IYC | 0.01886 | 0.866 | 0.863 | -0.0005 | 1.0310 | 0.1502 | -0.0318 | 0.1490 | -0.1061 |